In [17]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pyspark import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [18]:
# Creating the spark session
#.config("spark.driver.extraClassPath", Jars) - Optional

Jars = "C:/Spark/spark-3.3.0-bin-hadoop3/jars/mssql-jdbc-10.2.1.jre8.jar; C:/Spark/spark-3.3.0-bin-hadoop3/jars/sqljdbc42.jar; C:/Spark/spark-3.3.0-bin-hadoop3/jars/sqljdbc_auth.dll"

spark = SparkSession.builder\
.appName("application one")\
.config("spark.master", "local[*]")\
.getOrCreate()

In [19]:
# Connection string to SQL SERVER

server_name = "jdbc:sqlserver://LOHITHGK"
database_name = "AdWorks"
url = server_name + ";" + "databaseName=" + database_name + ";" + "encrypt=true"+ ";" + "trustServerCertificate=true" + ";" + "integratedSecurity=true"

In [20]:
# File location and type
file_location = r"C:/Users/user/Desktop/Employee_output.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

In [21]:
# The applied options are for CSV files. For other file types, these will be ignored.
df = spark\
    .read\
    .format(file_type)\
    .option("inferSchema", infer_schema)\
    .option("header", first_row_is_header)\
    .option("sep", delimiter)\
    .load(file_location)

In [22]:
# create Temp view or Replace the existing view.
df.createOrReplaceTempView('Employee_output_temp')

In [23]:
# Query the temp View using Spark SQL.
HumanResources_Shift_f = spark.sql("""
select * 
from Employee_output_temp
""")

In [24]:
# Write the dataframe data to SQL db table. 
# If the table is not existed, then it would create the table and then perform write operation.
# .mode('append') - append the the data, .mode('overwrite') - overwrite the exisiting data.

HumanResources_Shift_f.write\
    .format('jdbc')\
    .mode('overwrite')\
    .option("truncate","true")\
    .option('url', url)\
    .option('dbtable', 'AdWorks.dbo.Employee_output_test')\
    .save()